# Hashtables manually

Some info on the topic:

* https://www.geeksforgeeks.org/hashing-set-2-separate-chaining/
* https://www.geeksforgeeks.org/hashing-set-3-open-addressing/

In [49]:
# Hash functions

def multiply_shift(x):
    WORDLENGTH = 32
    TARGET_SIZE = 8
    return (2654435769*x & (2**WORDLENGTH-1)) >> (WORDLENGTH-TARGET_SIZE)

print([multiply_shift(i) for i in range(100)])

[0, 158, 60, 218, 120, 23, 181, 83, 241, 143, 46, 204, 106, 8, 167, 69, 227, 129, 31, 190, 92, 250, 152, 54, 213, 115, 17, 175, 78, 236, 138, 40, 198, 101, 3, 161, 63, 222, 124, 26, 184, 86, 245, 147, 49, 207, 109, 12, 170, 72, 230, 133, 35, 193, 95, 253, 156, 58, 216, 118, 21, 179, 81, 239, 141, 44, 202, 104, 6, 164, 67, 225, 127, 29, 188, 90, 248, 150, 52, 211, 113, 15, 173, 75, 234, 136, 38, 196, 99, 1, 159, 61, 219, 122, 24, 182, 84, 243, 145, 47]


In [1]:
class Htsc:
    """Hash table with separate chaining."""
    class Node:
        """Linked list node."""
        def __init__(self,key=None,val=None):
            self.key = key
            self.val = val
            self.next = None
        
        def __str__(self):
            out = str(self.key) + ':' + str(self.val)
            if self.next is None:
                return out
            return out + ' → ' + str(self.next)
        
        def push(self,key,val):
            if self.key is None:
                self.key = key
                self.val = val
            elif self.key==key:
                self.val = val
            elif self.next is None:
                self.next = Htsc.Node(key,val) # No other way to reference outer class
            else:
                self.next.push(key,val)
                
        def read(self,key):
            if self.key == key:
                return self.val
            elif self.next is None:
                return None
            return self.next.read(key)
            
        def rm(self,key):
            if self.key == key:
                return self.next
            if self.next is not None:
                self.next = self.next.rm(key) # Recurrent elimination
                return self
            raise KeyError('Invalid key')
    
    def __init__(self,m=11):
        self.m = m # Some prime
        self.array = [None]*self.m
    
    def hf(self,key):
        """Hash function"""
        if type(key) is int:
            return key % self.m
        if type(key) is str:
            out = 0
            for s in list(key):
                out = out*127 + ord(s) # 127 is prime, and comparable to ord(z)
            return out % self.m        
    
    def add(self,key,val):
        """Add a value."""
        h = self.hf(key)
        if self.array[h] is None:
            self.array[h] = Htsc.Node(key,val)
        else:
            self.array[h].push(key,val)
        
    def get(self,key):
        """Read a value."""
        h = self.hf(key)
        if self.array[h] is None:
            return None
        return self.array[h].read(key)
    
    def rm(self,key):
        h = self.hf(key)
        if self.array[h] is None:
            raise KeyError('Invalid key')
        self.array[h] = self.array[h].rm(key)
    
    def print(self):
        for n in self.array:
            print(n)
    
# Testing
ht = Htsc(m=3)
ht.add('cat',5)
ht.add('dog',100)
ht.add(5,7)  # Writing on the same place, but with a different key
ht.add(5,12) # Same key
ht.print()
print()
print(ht.get(5))
print(ht.get(7))
print()
ht.rm('dog')
ht.print()
#ht.rm('cow') # Returns an error

cat:5
None
dog:100 → 5:12

12
None

cat:5
None
5:12


In [67]:
for i in range(10): # Testing overlow (this hashmap shouldn't overflow)
    ht.add(i,i)
ht.print()

cat:5 → 0:0 → 3:3 → 6:6 → 9:9
1:1 → 4:4 → 7:7
5:5 → 2:2 → 8:8


In [2]:
class Htlp:
    """Hash table with linear probing."""
    def __init__(self,m=7):
        self.m = m  # Size
        self.a = [None]*self.m
        
    def __str__(self):
        return ' '.join([str(a) for a in self.a])
    
    def hf(self,key):
        """Hash function"""
        if type(key) is int:
            return key % self.m
        if type(key) is str:
            out = 0
            for s in list(key):
                out = out*127 + ord(s) # 127 is prime, and comparable to ord(z)
            return out % self.m    
    
    def add(self,key,val):
        i = self.hf(key)
        # print(key,'->',i)
        j = 0
        while j < self.m:
            k = (i+j) % self.m
            if self.a[k] in [None,'del']: # None for pristine, del for deleted (special magic value)
                self.a[k] = (key,val)
                return
            if self.a[k][0]==key: # At this point we know it's not None
                self.a[k] = (key,val)
                return
            j += 1
        raise MemoryError('Hashmap overflow.')      # We're not looping to the beginning, but we could
        
    def find(self,key):
        i = self.hf(key)
        j = 0
        while j<self.m:
            k = (i+j) % self.m
            if self.a[k] is None:
                return None
            if self.a[k]=='del': # We should continue on dels, as if they were normal values
                j += 1
                continue
            if self.a[k][0]==key:
                return k
            j += 1 # Strictly speaking 'del' above can be tucked into this case (no need for a separate 'if')
        return None
    
    def get(self,key):
        k = self.find(key)
        if k is None: return None
        return self.a[k][1]
    
    def rm(self,key):
        k = self.find(key)
        if k is None: raise KeyError('Invalid key')
        self.a[k] = 'del'
    
ht = Htlp(m=7)
ht.add('cat',5)
ht.add('dog',100)
ht.add(6,7)  # Writing on the same place, but with a different key
ht.add(6,12) # Same key
ht.add(13,'Collision')
print(ht)
print(ht.get(6))
print(ht.get('dog'))
print(ht.get('cow'))
ht.rm(6)
print(ht)
print(ht.get(13))

(6, 12) (13, 'Collision') None None ('cat', 5) None ('dog', 100)
12
100
None
del (13, 'Collision') None None ('cat', 5) None ('dog', 100)
Collision


In [53]:
for i in range(10): # This should overflow the hashmap
    ht.add(i,i)

MemoryError: Hashmap overflow.

In [35]:
def divisors(k):
    """A quick silly tool to find prime numbers."""
    return [i for i in range(2,k-1) if k % i==0]

print(divisors(60))
print(divisors(35769))

[2, 3, 4, 5, 6, 10, 12, 15, 20, 30]
[3, 11923]
